This notebook walks through how to make graphs in this repo. You can copy this example.ipynb and rename it for your own graphing work. <b> Please do each graph in a seperate notebook!</b>

1. Import the appropriate packages. We'll use plotly.graph_objects for making all of our graphs.

In [2]:
import datetime as dt
import plotly.graph_objects as go
import plotly.io as pio
import pandas as pd
import utils.utils as ut
import utils.graph_templates

from fredapi import Fred

2. We'll get data using the fredapi package. Set the path to the text file with your API key

In [3]:
# Call the graph the exact same thing as its notebook (minus the ipynb suffix)
GRAPH_NAME = "Unemployment Rate by Race"

API_KEY_PATH = ut.get_repo_root() / "fred_api_key.txt" 

fred = Fred(api_key_file = API_KEY_PATH)

3. Set the fed_2025 template as default

In [4]:
pio.templates.default = 'fed_2025'

# Now is a good time to set the path to the graph output folder!
GRAPH_OUTPUT_PATH = ut.get_repo_root() / "figures"

4. Use the fredapi to get the data and prepare it for graphing. Documentation on the optional parameters that can be passed to the get_series called are found here (the documentation in fredapi is out of date). 

https://fred.stlouisfed.org/docs/api/fred/series_observations.html#Description

If you get data from somewhere else thats fine too! Put the raw csv in the "raw_data" folder and read it in here. Make sure not to edit the raw data, just transform and graph it.

In [5]:
# The get_series_info method may be useful
fred.get_series_info(series_id="CPIAUCSL")

# It's good practice to store the series codes in a dictionary with their names
series_codes = {
    "Unemployment Rate - African American": "LNU04000006",
    "Unemployment Rate - White":"LNS14000003",
    "Unemployment Rate - Asian": "LNU04032183",
    "Unemployment Rate - Hispanic or Latino":"LNS14000009"
}

In [16]:
today = dt.date.today()

un_black=fred.get_series(
    series_id=series_codes["Unemployment Rate - African American"],
    observation_start=dt.date(2019, 1, 1),
    observation_end=today,
    frequency='m',
).rename("Unemployment Rate - African American")

un_white=fred.get_series(
    series_id=series_codes["Unemployment Rate - White"],
    observation_start=dt.date(2019, 1, 1),
    observation_end=today,
    frequency='m'
).rename("Unemployment Rate - White")

un_asian=fred.get_series(
    series_id=series_codes["Unemployment Rate - Asian"],
    observation_start=dt.date(2019, 1, 1),
    observation_end=today,
    frequency='m'
).rename("Unemployment Rate - Asian")

un_latino=fred.get_series(
    series_id=series_codes["Unemployment Rate - Hispanic or Latino"],
    observation_start=dt.date(2019, 1, 1),
    observation_end=today,
    frequency='m'
).rename("Unemployment Rate - Hispanic or Latino")

joined_df = pd.concat(
    [un_black, un_white,un_asian,un_latino], # srs to join
    axis=1, # multiple observations per index entry
    join='inner' # only include observations where none of the 3 are missing
)

# Naming the index makes it easier tot title
joined_df.index.name = "Date"

joined_df.tail()

,Unemployment Rate - African American,Unemployment Rate - White,Unemployment Rate - Asian,Unemployment Rate - Hispanic or Latino
Date,,,,
2025-04-01,5.8,3.8,2.8,5.2
2025-05-01,5.9,3.8,3.4,5.1
2025-06-01,7.2,3.6,3.9,4.8
2025-07-01,7.9,3.7,4.1,5.0
2025-08-01,8.0,3.7,3.7,5.3


5. Now that all our data is ready, make the graph and have it save itself as a .html file to graph_output whenver the notebooks is rerun. The name of the file should exactly match the notebook name. For instance, this file "example.ipynb" produces the graph "example.html." Nice work, you made a graph! 

In [73]:
# First make the figure
fig = go.Figure()

# Loop the columns of the dataframe and plot each as a separate trace
for col in joined_df.columns:
    fig.add_trace(
        go.Scatter(
            x=joined_df.index,
            y=joined_df[col],
            mode='lines',
            name=col
        )
    )

# Update the titles, using the html tage <sup> for a subtitle 
fig.update_layout(
    title = dict(text = 'Unemployment Rate by Race <br><sup>Monthly, Not Seasonally Adjusted </sup>'),
    xaxis_title="Date",
    yaxis_title="Unemployment Rate (%)",
    # Some new legend settings to move it closer to the title
    legend=dict(
        y=1,
        x=1,
        yanchor='top', # Some anchoring to get the necessary aesthetic.
        xanchor='center',
        orientation='h'
    )
)


# This is graph specific, but here we want the y-axis to be percent signs 
fig.update_yaxes(
    tickformat="0.2f%",
    ticksuffix="%"
)

# Again, graph specific, we have a mutliyear series and want tick marks to be years
fig.update_xaxes(
    type='date',
    tickformat='%Y',
)

# ut.add_end_labels(fig = fig)

# Show our figure (Dimensions may be off on different screen sizes)
fig.show()

# This should be the same for EVERY GRAPH!
# Save it to the graph_output folder with the name matching the file, as HTML
fig.write_html(GRAPH_OUTPUT_PATH / f"{GRAPH_NAME}.html")